In [1]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade import plotter
import numpy as np
from pyalgotrade.barfeed import quandlfeed
import datetime
from pyalgotrade.technical import ma
from pyalgotrade.technical import cross
filename = "nifty5min.csv"
import pytz
firstDate = datetime.datetime(2012,5,30,9,15,0,0,pytz.utc)
endDate = datetime.datetime(2016,12,9,15,25,0,0,pytz.utc)
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [2]:
class SMACrossOver(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed, 100000)
        self.__instrument = instrument
        self.__position = None
        # We'll use adjusted close values instead of regular close values.
        #self.setUseAdjustedValues(False)
        #self.__prices = feed[instrument].getPriceDataSeries()
        #self.__sma = ma.SMA(self.__prices, smaPeriod)
        self.__sma10 = ma.SMA(feed[instrument].getCloseDataSeries(), 10)
        self.__sma20 = ma.SMA(feed[instrument].getCloseDataSeries(), 20)

    def getSMA(self):
        return self.__sma10
    
    def getSMA(self):
        return self.__sma20
    
    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("sell at %.2f" % (execInfo.getPrice()))
        
    

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("buy at %.2f" % (execInfo.getPrice()))
        self.__position = None
        

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # If a position was not opened, check if we should enter a long position.
        bar = bars[self.__instrument]
        if self.__position is None :
            if cross.cross_below(self.__sma20,self.__sma10) > 0 and (bar.getDateTime().hour>10 and bar.getDateTime().hour<14):
                shares = int(self.getBroker().getCash() * 0.9 / bars[self.__instrument].getPrice())
                #self.info(bar.getDateTime()) 
                # Enter a buy market order. The order is good till canceled.
                self.__position = self.enterShort(self.__instrument, shares, True)
        # Check if we have to exit the position.
        elif not self.__position.exitActive() and cross.cross_above(self.__sma20,self.__sma10) > 0:
            self.__position.exitMarket()

In [3]:
def run_strategy():
    # Load the bar feed from the CSV file
    #feed = quandlfeed.Feed()
    #feed.addBarsFromCSV("yes_bank", "yes_bankk.csv")

    # Evaluate the strategy with the feed.
  
    feed = csvfeed.GenericBarFeed(Frequency.MINUTE,pytz.utc)
#feed = quandlfeed.Feed()
    feed.setBarFilter(csvfeed.DateRangeFilter(firstDate,endDate))
    feed.addBarsFromCSV('bank_nifty_15min',filename)

    #myStrategy = MyStrategy(feed, "bank_nifty")
    #myStrategy.run()
    crossover = SMACrossOver(feed,"bank_nifty_15min")
    crossover.run()
    print("Final portfolio value: %.2f" % crossover.getBroker().getEquity())

In [4]:
run_strategy()

2012-05-30 11:45:00+00:00 strategy [INFO] sell at 9540.00
2012-05-30 12:30:00+00:00 strategy [INFO] buy at 9510.55
2012-05-30 14:00:00+00:00 strategy [INFO] sell at 9516.00
2012-05-30 15:00:00+00:00 strategy [INFO] buy at 9486.00
2012-05-31 12:00:00+00:00 strategy [INFO] sell at 9390.25
2012-05-31 13:50:00+00:00 strategy [INFO] buy at 9432.00
2012-05-31 14:00:00+00:00 strategy [INFO] sell at 9440.00
2012-06-01 09:20:00+00:00 strategy [INFO] buy at 9413.00
2012-06-01 11:20:00+00:00 strategy [INFO] sell at 9425.40
2012-06-01 11:40:00+00:00 strategy [INFO] buy at 9339.00
2012-06-01 13:40:00+00:00 strategy [INFO] sell at 9291.35
2012-06-01 13:55:00+00:00 strategy [INFO] buy at 9251.00
2012-06-04 13:15:00+00:00 strategy [INFO] sell at 9194.45
2012-06-05 11:20:00+00:00 strategy [INFO] buy at 9414.90
2012-06-05 11:45:00+00:00 strategy [INFO] sell at 9480.00
2012-06-05 13:00:00+00:00 strategy [INFO] buy at 9418.60
2012-06-06 12:15:00+00:00 strategy [INFO] sell at 9569.90
2012-06-06 13:00:00+00

2012-09-06 15:05:00+00:00 strategy [INFO] buy at 9947.50
2012-09-07 12:50:00+00:00 strategy [INFO] sell at 10118.20
2012-09-07 14:10:00+00:00 strategy [INFO] buy at 10124.20
2012-09-10 12:35:00+00:00 strategy [INFO] sell at 10152.00
2012-09-10 13:20:00+00:00 strategy [INFO] buy at 10096.55
2012-09-11 11:30:00+00:00 strategy [INFO] sell at 10056.00
2012-09-11 13:50:00+00:00 strategy [INFO] buy at 10102.00
2012-09-12 12:15:00+00:00 strategy [INFO] sell at 10232.20
2012-09-12 12:50:00+00:00 strategy [INFO] buy at 10186.50
2012-09-14 12:35:00+00:00 strategy [INFO] sell at 10610.00
2012-09-14 15:20:00+00:00 strategy [INFO] buy at 10659.00
2012-09-17 13:15:00+00:00 strategy [INFO] sell at 10904.90
2012-09-18 09:15:00+00:00 strategy [INFO] buy at 10950.00
2012-09-18 12:05:00+00:00 strategy [INFO] sell at 11011.60
2012-09-18 13:20:00+00:00 strategy [INFO] buy at 11023.55
2012-09-18 13:50:00+00:00 strategy [INFO] sell at 11036.95
2012-09-20 09:25:00+00:00 strategy [INFO] buy at 10952.00
2012-09

2012-12-20 13:35:00+00:00 strategy [INFO] sell at 12503.20
2012-12-21 09:20:00+00:00 strategy [INFO] buy at 12455.95
2012-12-21 13:30:00+00:00 strategy [INFO] sell at 12429.35
2012-12-21 14:40:00+00:00 strategy [INFO] buy at 12413.90
2012-12-24 11:20:00+00:00 strategy [INFO] sell at 12306.55
2012-12-24 11:45:00+00:00 strategy [INFO] buy at 12311.00
2012-12-24 12:10:00+00:00 strategy [INFO] sell at 12334.00
2012-12-24 13:00:00+00:00 strategy [INFO] buy at 12314.00
2012-12-24 13:40:00+00:00 strategy [INFO] sell at 12350.15
2012-12-24 14:25:00+00:00 strategy [INFO] buy at 12308.00
2012-12-26 11:05:00+00:00 strategy [INFO] sell at 12354.50
2012-12-26 14:10:00+00:00 strategy [INFO] buy at 12516.10
2012-12-27 11:35:00+00:00 strategy [INFO] sell at 12503.75
2012-12-27 12:00:00+00:00 strategy [INFO] buy at 12495.25
2012-12-28 13:55:00+00:00 strategy [INFO] sell at 12542.10
2012-12-28 14:05:00+00:00 strategy [INFO] buy at 12545.00
2012-12-31 13:40:00+00:00 strategy [INFO] sell at 12571.90
2012-

2013-03-13 09:20:00+00:00 strategy [INFO] buy at 12021.00
2013-03-13 11:30:00+00:00 strategy [INFO] sell at 11981.00
2013-03-13 12:25:00+00:00 strategy [INFO] buy at 11967.10
2013-03-13 13:50:00+00:00 strategy [INFO] sell at 11958.35
2013-03-13 14:15:00+00:00 strategy [INFO] buy at 11965.00
2013-03-15 13:45:00+00:00 strategy [INFO] sell at 11940.00
2013-03-15 15:00:00+00:00 strategy [INFO] buy at 11858.25
2013-03-18 11:40:00+00:00 strategy [INFO] sell at 11725.00
2013-03-18 12:00:00+00:00 strategy [INFO] buy at 11716.95
2013-03-18 12:25:00+00:00 strategy [INFO] sell at 11746.50
2013-03-19 09:25:00+00:00 strategy [INFO] buy at 11843.50
2013-03-19 12:45:00+00:00 strategy [INFO] sell at 11576.00
2013-03-19 13:10:00+00:00 strategy [INFO] buy at 11561.35
2013-03-20 12:55:00+00:00 strategy [INFO] sell at 11447.50
2013-03-20 13:50:00+00:00 strategy [INFO] buy at 11380.55
2013-03-22 12:00:00+00:00 strategy [INFO] sell at 11301.80
2013-03-22 12:50:00+00:00 strategy [INFO] buy at 11275.10
2013-0

2013-06-18 12:10:00+00:00 strategy [INFO] buy at 11907.95
2013-06-18 13:40:00+00:00 strategy [INFO] sell at 11886.55
2013-06-18 15:00:00+00:00 strategy [INFO] buy at 11880.00
2013-06-19 11:10:00+00:00 strategy [INFO] sell at 11790.05
2013-06-19 12:50:00+00:00 strategy [INFO] buy at 11798.50
2013-06-19 13:50:00+00:00 strategy [INFO] sell at 11840.10
2013-06-20 09:20:00+00:00 strategy [INFO] buy at 11565.00
2013-06-20 13:20:00+00:00 strategy [INFO] sell at 11473.05
2013-06-20 13:40:00+00:00 strategy [INFO] buy at 11427.80
2013-06-21 12:15:00+00:00 strategy [INFO] sell at 11373.50
2013-06-21 13:45:00+00:00 strategy [INFO] buy at 11364.75
2013-06-24 12:55:00+00:00 strategy [INFO] sell at 11221.00
2013-06-24 13:50:00+00:00 strategy [INFO] buy at 11198.95
2013-06-25 11:45:00+00:00 strategy [INFO] sell at 11203.55
2013-06-25 14:50:00+00:00 strategy [INFO] buy at 11251.50
2013-06-26 11:55:00+00:00 strategy [INFO] sell at 11218.75
2013-06-26 14:20:00+00:00 strategy [INFO] buy at 11190.15
2013-0

...

2016-02-18 14:05:00+00:00 strategy [INFO] buy at 14189.30
2016-02-19 11:15:00+00:00 strategy [INFO] sell at 14295.45
2016-02-19 12:30:00+00:00 strategy [INFO] buy at 14304.85
2016-02-22 13:15:00+00:00 strategy [INFO] sell at 14364.80
2016-02-22 14:45:00+00:00 strategy [INFO] buy at 14371.15
2016-02-23 11:40:00+00:00 strategy [INFO] sell at 14137.20
2016-02-23 12:10:00+00:00 strategy [INFO] buy at 14120.25
2016-02-24 12:10:00+00:00 strategy [INFO] sell at 13858.00
2016-02-24 14:10:00+00:00 strategy [INFO] buy at 13894.00
2016-02-25 13:40:00+00:00 strategy [INFO] sell at 13726.80
2016-02-25 14:30:00+00:00 strategy [INFO] buy at 13651.00
2016-02-26 12:20:00+00:00 strategy [INFO] sell at 13718.00
2016-02-26 15:10:00+00:00 strategy [INFO] buy at 13842.45
2016-02-29 13:15:00+00:00 strategy [INFO] sell at 14137.25
2016-02-29 14:50:00+00:00 strategy [INFO] buy at 13984.00
2016-03-01 11:45:00+00:00 strategy [INFO] sell at 14291.95
2016-03-02 11:30:00+00:00 strategy [INFO] buy at 15000.05
2016-0

2016-06-02 12:55:00+00:00 strategy [INFO] sell at 17436.65
2016-06-03 10:55:00+00:00 strategy [INFO] buy at 17664.00
2016-06-03 11:30:00+00:00 strategy [INFO] sell at 17682.00
2016-06-03 11:35:00+00:00 strategy [INFO] buy at 17687.50
2016-06-03 11:45:00+00:00 strategy [INFO] sell at 17694.60
2016-06-03 13:00:00+00:00 strategy [INFO] buy at 17708.00
2016-06-03 13:05:00+00:00 strategy [INFO] sell at 17708.00
2016-06-03 13:55:00+00:00 strategy [INFO] buy at 17690.00
2016-06-06 12:30:00+00:00 strategy [INFO] sell at 17702.95
2016-06-06 13:50:00+00:00 strategy [INFO] buy at 17698.45
2016-06-07 11:55:00+00:00 strategy [INFO] sell at 17769.95
2016-06-07 14:50:00+00:00 strategy [INFO] buy at 17879.90
2016-06-08 13:45:00+00:00 strategy [INFO] sell at 17907.95
2016-06-08 14:15:00+00:00 strategy [INFO] buy at 17890.65
2016-06-09 11:10:00+00:00 strategy [INFO] sell at 17878.00
2016-06-09 12:15:00+00:00 strategy [INFO] buy at 17880.00
2016-06-09 12:20:00+00:00 strategy [INFO] sell at 17910.10
2016-

2016-09-07 14:50:00+00:00 strategy [INFO] buy at 20540.25
2016-09-08 13:30:00+00:00 strategy [INFO] sell at 20474.50
2016-09-08 14:15:00+00:00 strategy [INFO] buy at 20487.80
2016-09-09 11:50:00+00:00 strategy [INFO] sell at 20290.15
2016-09-09 12:00:00+00:00 strategy [INFO] buy at 20281.05
2016-09-09 12:30:00+00:00 strategy [INFO] sell at 20276.50
2016-09-09 12:40:00+00:00 strategy [INFO] buy at 20270.95
2016-09-09 12:45:00+00:00 strategy [INFO] sell at 20344.65
2016-09-09 14:55:00+00:00 strategy [INFO] buy at 20353.95
2016-09-12 11:55:00+00:00 strategy [INFO] sell at 19915.00
2016-09-12 12:50:00+00:00 strategy [INFO] buy at 19863.05
2016-09-14 11:15:00+00:00 strategy [INFO] sell at 19920.10
2016-09-14 13:30:00+00:00 strategy [INFO] buy at 19965.05
2016-09-15 11:45:00+00:00 strategy [INFO] sell at 19882.25
2016-09-15 12:25:00+00:00 strategy [INFO] buy at 19837.95
2016-09-15 13:20:00+00:00 strategy [INFO] sell at 19884.90
2016-09-16 11:35:00+00:00 strategy [INFO] buy at 20251.00
2016-0

2016-12-06 11:05:00+00:00 strategy [INFO] sell at 18466.05
2016-12-06 12:55:00+00:00 strategy [INFO] buy at 18505.00
2016-12-06 13:35:00+00:00 strategy [INFO] sell at 18506.85
2016-12-06 14:00:00+00:00 strategy [INFO] buy at 18509.90
2016-12-07 13:00:00+00:00 strategy [INFO] sell at 18600.60
2016-12-07 14:30:00+00:00 strategy [INFO] buy at 18601.00
2016-12-08 12:10:00+00:00 strategy [INFO] sell at 18539.75
2016-12-08 13:20:00+00:00 strategy [INFO] buy at 18530.00
2016-12-08 13:55:00+00:00 strategy [INFO] sell at 18571.05
2016-12-08 15:05:00+00:00 strategy [INFO] buy at 18578.00


Final portfolio value: 95543.30
